#Rafael Souza de Barros
## Objetivo e perguntas do projeto MVP - Engenharia de dados
### Pós-graduação em ciência de dados e _analytics_ - PUC-RIO


O presente projeto foi elaborado para fins de conclusão de módulo (_sprint_) de engenhraria de dados do curso de pós-graduação em ciência de dados e _analytics_ da PUC-RIO e tem como objetivo consumir os dados disponibilizados de forma pública e aberta pelo Supremo Tribunal Federal - STF, por meio do site "Corte Aberta", disponível em (https://portal.stf.jus.br/hotsites/corteaberta/).

Decidi utilizar nesse projeto o conjunto de dados sobre distribuições e registros e sobre decisões (monocráticas ou colegiadas), filtrados para o ano de 2025, por uma questão de limitação de recursos. 

Com esses dados, objetiva-se identificar possíveis tendências de julgamento da Suprema Corte em função dos temas, do Tribunal de origem e da parte, respondendo as seguintes perguntas:

1. Quais e quantos processos foram distribuídos no ano de 2025?
2. É possível identificar as pessoas e entidades com maior quantidade de recursos e o polo processual que elas ocupam?
3. Qual o resultado dos julgamentos desses processos? 
4. Existe alguma tendência nos julgamentos (favoráveis ou desfavoráveis) por tema, origem ou pessoa/entidade?
5. Qual tempo médio de tramitação dos recursos por origem e por Ministro Relator dos processos?

Para alcançar a resposta dessas perguntas serão seguidas as seguintes etapas:

1. Carregar no _Databricks_ as tabelas obtidas no site (https://portal.stf.jus.br/hotsites/corteaberta/)
2. Construir catálogo de dados das tabelas ingeridas
3. Manipular/limpar os dados (definir tipos de dados nas tabelas, juntar colunas Classe e Nº Processo, fazer join das 2 tabelas, etc.)
4. Identificar valores que faltam nas colunas e definir se é um problema que impede a resposta das perguntas iniciais
5. Analisar os dados e responder as perguntas propostas
6. Auto avaliação 




#### 1. Carregamento no _Databricks_ das tabelas obtidas no site (https://portal.stf.jus.br/hotsites/corteaberta/) - camada Bronze

In [0]:
# Carregando as funções

from pyspark.sql.functions import col, to_date, when, trim, regexp_replace, coalesce, lit, concat_ws, sum, count
from pyspark.sql.types import StringType
from pyspark.sql import DataFrame


# 1. Ler a tabelas carregadas no catálogo

dist_raw = spark.table("distribuicoes_2025_stf")

dec_raw = spark.table("decisoes_2025_stf")

 


In [0]:
dist_raw.printSchema()

In [0]:
dec_raw.printSchema()

#### 2. Construir catálogo de dados das tabelas ingeridas

#### 2. Manipular/limpar os dados (definir tipos de dados nas tabelas, juntar colunas Classe e Nº Processo, fazer join das 2 tabelas, etc.) - camada Prata

##### 2.1 Ajustando tipos de dados das colunas

Identificou-se pelas tabelas acima, que os dados ingeridos não tiveram corretamente classificados os tipos dos dados, como as colunas "DAta de Autuação" e "Data Baixa" que foram consideradas como _string_.

Assim, foi necessário identificar manualmente esses tipos. Além disso, algumas colunas de datas vieram no formato data/hora e outras apenas no formata data (dd/mm/yyyy), sendo necessária a manipulação para utilização desses dados.

In [0]:


dist_typed = (
    dist_raw
    .withColumn("Nº do processo", col("Nº do processo").cast("long"))
    .withColumn(
        "Data da autuação",
        when(trim(col("Data da autuação")) == "-", None)
        .otherwise(to_date(col("Data da autuação"), "dd/MM/yyyy"))
    )
    .withColumn(
        "Data da baixa",
        when(trim(col("Data da baixa")) == "-", None)
        .otherwise(to_date(col("Data da baixa"), "dd/MM/yyyy"))
    )
    .withColumn(
        "Data do andamento",
        when(trim(col("Data do andamento")) == "-", None)
        .otherwise(to_date(col("Data do andamento"), "dd/MM/yyyy"))
    )
    .withColumn(
        "Em tramitação",
        when(trim(col("Em tramitação")) == "Sim", True)
        .when(trim(col("Em tramitação")) == "Não", False)
        .otherwise(None)
        .cast("boolean")
    )
    .withColumn(
        "Indicador de substituição ou redistribuição",
        when(trim(col("Indicador de substituição ou redistribuição")) == "Sim", True)
        .when(trim(col("Indicador de substituição ou redistribuição")) == "Não", False)
        .otherwise(None)
        .cast("boolean")
    )
    .withColumn(
    "processo",
    concat_ws(" ", col("Classe"), col("Nº do processo"))
)
)



def normaliza_data(c):
    return when(trim(col(c)).isin("*NI*", "-", ""), lit(None)) \
        .otherwise(
            to_date(
                regexp_replace(col(c), r"\s.*", ""),
                "dd/MM/yyyy"
            )
        )

dec_typed = (
    dec_raw
    .withColumn("idFatoDecisao", col("idFatoDecisao").cast("long"))
    .withColumn("Ano da decisão", col("Ano da decisão").cast("int"))
    .withColumn("Data de autuação", normaliza_data("Data de autuação"))
    .withColumn("Data baixa", normaliza_data("Data baixa"))
    .withColumn("Data da decisão", normaliza_data("Data da decisão"))
    .withColumn(
        "Indicador de tramitação",
        when(trim(col("Indicador de tramitação")) == "Sim", True)
        .when(trim(col("Indicador de tramitação")) == "Não", False)
        .otherwise(None)
        .cast("boolean")
    )
)

##### 3.2 Renomeando colunas e salvando em formato Delta na camada _Silver_

Para salvar os conjuntos de dados em formato Delta, foi constatado que os nomes das colunas não eram suportados por esse formato, tendo sido preciso a alteração dos nomes para formas compatíveis.

Também se identiicou que alguns valores inexistentes estavam preenchidos com símbolos, o que dificulta a sua contagem como nulos, tendo sido feita, então, a explicitação desses valores nulos, para fins de avaliação das qualidades dos dados.

Por fim, foi criada a base de dados "silver" e salvos os dados limpos em formato Delta.

In [0]:


# Renomear colunas para nomes válidos em Delta

dec = dec_typed

rename_map_dec = {
    "idFatoDecisao": "id_fato_decisao",
    "Processo": "processo",
    "Relator atual": "relator_atual",
    "Meio Processo": "meio_processo",
    "Origem decisão": "origem_decisao",
    "Ambiente julgamento": "ambiente_julgamento",
    "Data de autuação": "data_autuacao",
    "Data baixa": "data_baixa",
    "Indicador colegiado": "indicador_colegiado",
    "Ano da decisão": "ano_decisao",
    "Data da decisão": "data_decisao",
    "Tipo decisão": "tipo_decisao",
    "Andamento decisão": "andamento_decisao",
    "Observação do andamento": "observacao_andamento",
    "Ramo direito": "ramo_direito",
    "Assuntos do processo": "assuntos_processo",
    "Indicador de tramitação": "indicador_tramitacao",
    "Órgão julgador": "orgao_julgador",
}

for old, new in rename_map_dec.items():
    dec = dec.withColumnRenamed(old, new)



dist = dist_typed

rename_map_dist = {
    "Tipo de andamento": "tipo_andamento",
    "Classe": "classe",
    "Nº do processo": "num_processo",
    "Link": "link",
    "Ministro(a)": "ministro",
    "Última localização": "ultima_localizacao",
    "Data da autuação": "data_autuacao",
    "Data da baixa": "data_baixa",
    "Em tramitação": "em_tramitacao",
    "Grupo origem": "grupo_origem",
    "Meio processo": "meio_processo",
    "Data do andamento": "data_andamento",
    "Andamento": "andamento",
    "Subgrupo do andamento": "subgrupo_andamento",
    "Indicador de substituição ou redistribuição": "ind_substituicao_redistribuicao",
    "Orgão origem": "orgao_origem",
    "Procedência": "procedencia",
    "Ramo do direito": "ramo_direito",
    "Assunto completo": "assunto_completo",
    "Polo ativo": "polo_ativo",
    "Advogado polo ativo": "advogado_polo_ativo",
    "Polo passivo": "polo_passivo",
    "Advogado polo passivo": "advogado_polo_passivo",
    "processo": "processo"
}

for old, new in rename_map_dist.items():
    dist = dist.withColumnRenamed(old, new)


# Explicitando valores nulos para as linhas preenchidas com "-", "", "*NI*", "N/I", "NA"

dist_corrigido = dist

for campo in dist.schema.fields:
    if isinstance(campo.dataType, StringType):
        dist_corrigido = dist_corrigido.withColumn(
            campo.name,
            when(trim(col(campo.name)).isin("-", "", "*NI*", "N/I", "NA"), None)
            .otherwise(col(campo.name))
        )

dec_corrigido = dec

for campo in dec.schema.fields:
    if isinstance(campo.dataType, StringType):
        dec_corrigido = dec_corrigido.withColumn(
            campo.name,
            when(trim(col(campo.name)).isin("-", "", "*NI*", "N/I", "NA"), None)
            .otherwise(col(campo.name))
        )

# Gravando Silver em Delta

spark.sql("CREATE DATABASE IF NOT EXISTS silver")


dec_corrigido.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.decisoes2025stf")


dist_corrigido.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.distribuicoes2025stf")    




In [0]:
dist = spark.table("silver.distribuicoes2025stf")

dist.printSchema()

In [0]:
dec = spark.table("silver.decisoes2025stf")

dec.printSchema()

#### 4. Avaliar a qualidade dos dados e a existência de problemas que impeçam a resposta das perguntas

##### 4.1 Quantidade de nulos na tabela de distribuições e registros

In [0]:

dist_null = spark.table("silver.distribuicoes2025stf")

perfil = []

total = dist_null.count()

for field in dist_null.schema.fields:
    nome = field.name
    tipo = field.dataType.simpleString()
    
    qtd_nulos = dist_null.select(
        sum(col(nome).isNull().cast("int")).alias("nulos")
    ).collect()[0]["nulos"]

    perfil.append((nome, tipo, total, qtd_nulos, total - qtd_nulos))

perfil_dist_null = spark.createDataFrame(
    perfil,
    ["coluna", "tipo", "total_registros", "qtd_nulos", "qtd_nao_nulos"]
)

perfil_dist_null.show( n=23, truncate=False)



Do dataframe acima precebe-se que a coluna "data_baixa" possui 18.414 observações com valores nulo, o que pode representar apenas que alguns processos distribuídos no ano de 2025 no STF ainda não tiveram julgamento definitivo na Corte e permanecem em tramitação.

Observando a quantidade de processos que possuem o valor "True" na coluna "em_tramitacao", conclui-se que essa é a hipótese explicativa mais provável, porque constam exatamente 18.414 processos que não possuem a data da baixa do processo e ainda estão em tramitação (em_tramitacao = True).

In [0]:
%sql
SELECT
    COUNT(processo) AS qtde_processos
FROM silver.distribuicoes2025stf
WHERE data_baixa IS NULL AND em_tramitacao = True;

Outros valores de nulos que chamam a atenção correspodem as colunas "polo_ativo", "advogado_polo_ativo", "polo_passivo" e "advogado_polo_passivo", que possuem quase todas as suas linhas sem valores correspondentes.

Essa situação impossibilita a completa resposta da questão 4, pois inviabiliza a identificação de possível tendência dos julgamentos por pessoa/entidade.

Assim, feita pesquisa em outros bancos de dados disponíveis publicamente na _interntet_, especialmente o DataJud do CNJ, observou-se que eles não fornecem dados sobre as partes dos processos, o que impossibilitou o tratamento desses dados. 

##### 4.2 Quantidade de nulos na tabela de decisões

In [0]:
dec_null = spark.table("silver.decisoes2025stf")

perfil = []

total = dec_null.count()

for field in dec_null.schema.fields:
    nome = field.name
    tipo = field.dataType.simpleString()
    
    qtd_nulos = dec_null.select(
        sum(col(nome).isNull().cast("int")).alias("nulos")
    ).collect()[0]["nulos"]

    perfil.append((nome, tipo, total, qtd_nulos, total - qtd_nulos))

perfil_df = spark.createDataFrame(
    perfil,
    ["coluna", "tipo", "total_registros", "qtd_nulos", "qtd_nao_nulos"]
)

perfil_df.show(truncate=False)


Analisando agora a quantidade de observações faltantes no conjunto de dados de decisões do STF em 2025, novamente, verifica-se uma quantidade quase idêntica na coluna "data_baixa", o que fortalece a hipótese de prosseguimento da tramitação do processo.

Sendo verificada, ainda, uma considerável quantidade de valores nulos na coluna "observacao_andamento" (57.152), que representa apenas uma complementação da coluna anterior "andamento_decisão", o que não deve gerar maiores dificuldades para as respostas das perguntas a serem respondidas.

##### 4.3 Criando novas medidas para resposta das perguntas iniciais e salvando dados na camada _Gold_

In [0]:
%sql

/* Cria a base de dados 'gold' */
CREATE DATABASE IF NOT EXISTS gold;


/* Cria visão com análise da quantidade de decisões por processo e o tempo de tramitação, inclusive relacionado a recursos interno */
CREATE OR REPLACE VIEW gold.vw_distribuicoes2025_decisoes AS
WITH dec_base AS (
    SELECT
        processo,
        data_decisao,
        tipo_decisao,
        relator_atual
    FROM silver.decisoes2025stf
    WHERE data_decisao IS NOT NULL
),

/* Agregado geral por processo */
dec_agg AS (
    SELECT
        processo,
        MIN(data_decisao) AS primeira_decisao,
        MAX(data_decisao) AS ultima_decisao,
        COUNT(*)          AS qtd_decisoes
    FROM dec_base
    GROUP BY processo
),

/* Dados da PRIMEIRA decisão (tipo_decisao e relator_atual) */
dec_primeira AS (
    SELECT
        b.processo,
        b.data_decisao  AS primeira_decisao,
        b.tipo_decisao  AS tipo_decisao_primeira,
        b.relator_atual AS relator_atual_primeira
    FROM dec_base b
    INNER JOIN dec_agg a
        ON a.processo = b.processo
       AND b.data_decisao = a.primeira_decisao
),

/* 1ª decisão de recurso interno POSTERIOR à primeira decisão */
dec_recurso AS (
    SELECT
        b.processo,
        MIN(b.data_decisao) AS segunda_decisao_recurso
    FROM dec_base b
    INNER JOIN dec_agg a
        ON a.processo = b.processo
    WHERE b.data_decisao > a.primeira_decisao
      AND b.tipo_decisao = 'Decisão em recurso interno'
    GROUP BY b.processo
)

SELECT
    d.*,

    /* decisões (com tipo/relator da primeira decisão) */
    p.primeira_decisao,
    p.tipo_decisao_primeira,
    p.relator_atual_primeira,
    r.segunda_decisao_recurso AS segunda_decisao,
    a.ultima_decisao,
    a.qtd_decisoes,

    CASE WHEN a.processo IS NOT NULL THEN TRUE ELSE FALSE END AS possui_decisao,

    /* 1) Tempo (dias) entre autuação e baixa */
    CASE
        WHEN d.data_baixa IS NULL OR d.data_autuacao IS NULL OR d.data_baixa < d.data_autuacao THEN NULL
        ELSE DATEDIFF(d.data_baixa, d.data_autuacao)
    END AS tempo_tramitacao,

    /* 3) Tempo (dias) entre autuação e 1ª decisão */
    CASE
        WHEN p.primeira_decisao IS NULL OR d.data_autuacao IS NULL THEN NULL
        ELSE DATEDIFF(p.primeira_decisao, d.data_autuacao)
    END AS tempo_analise_relator,

    /* 2) Tempo (dias) entre 1ª decisão e 1ª decisão de recurso interno posterior */
    CASE
        WHEN p.primeira_decisao IS NULL OR r.segunda_decisao_recurso IS NULL THEN NULL
        ELSE DATEDIFF(r.segunda_decisao_recurso, p.primeira_decisao)
    END AS tempo_recurso_interno

FROM silver.distribuicoes2025stf d
LEFT JOIN dec_agg a
    ON d.processo = a.processo
LEFT JOIN dec_primeira p
    ON d.processo = p.processo
LEFT JOIN dec_recurso r
    ON d.processo = r.processo;




/*Olhando as 20 primeira linhas da visão */
SELECT *
FROM gold.vw_distribuicoes2025_decisoes
HEAD;




#### 5. Analisar os dados e responder as perguntas propostas


##### 5.1 

##### 5.2 Criando visão de processos distribuídos em 2025 com ou sem decisão

###Quantidade total de processos distribuídos no STF no ano de 2025

In [0]:
%sql

SELECT
    COUNT(DISTINCT processo) AS qtde_processos
FROM silver.distribuicoes2025stf
WHERE YEAR(data_autuacao) = 2025;


###Quantidade de processos distribuídos no STF no ano de 2025 por classe processual

In [0]:
%sql

SELECT
    classe,
    COUNT(DISTINCT processo) AS qtde_processos
FROM silver.distribuicoes2025stf
GROUP BY classe
ORDER BY qtde_processos DESC;


###Quantidade de processos autuados em 2025 no STF que possuem decisão

Qualquer tipo de decisão (True) ou nenhuma decisão (False)

In [0]:
%sql

SELECT
    possui_decisao,
    COUNT(DISTINCT processo) AS qtde_processos
FROM gold.vw_distribuicoes2025_decisoes
WHERE year(data_autuacao) = 2025
GROUP BY possui_decisao;




Quantidade de processos distribuídos em 2025 com Decisão Final

In [0]:
%sql
SELECT COUNT(DISTINCT processo) AS processos_decididos
FROM silver.decisoes2025stf
WHERE tipo_decisao = 'Decisão Final' AND YEAR(data_autuacao) = 2025;


Quantidade de processos decididos que foram objeto de recurso interno

In [0]:
%sql
SELECT COUNT(DISTINCT processo) AS processos_com_recurso_interno
FROM gold.vw_distribuicoes2025_decisoes
WHERE tempo_recurso_interno IS NOT NULL
AND YEAR(data_autuacao) = 2025;


Tempo de tramitação para julgamento do recurso interno interposto

In [0]:
%sql


SELECT
    tempo_recurso_interno,
    COUNT(DISTINCT processo) AS qtde_processos
FROM gold.vw_distribuicoes2025_decisoes
WHERE tempo_recurso_interno IS NOT NULL
GROUP BY tempo_recurso_interno
ORDER BY qtde_processos DESC


In [0]:
import matplotlib.pyplot as plt

# Consulta SQL no Databricks
query = """
SELECT
    tempo_recurso_interno,
    COUNT(DISTINCT processo) AS qtde_processos
FROM gold.vw_distribuicoes2025_decisoes
WHERE tempo_recurso_interno IS NOT NULL
GROUP BY tempo_recurso_interno
ORDER BY qtde_processos DESC
"""

# Carregar os dados em um DataFrame do Spark
df = spark.sql(query)

# Converter para Pandas para facilitar o gráfico
df_pandas = df.toPandas()

# Dados para o gráfico
x = df_pandas['tempo_recurso_interno']  # Eixo X: Tempo do recurso interno
y = df_pandas['qtde_processos']  # Eixo Y: Quantidade de processos
size = y  # Tamanho das bolhas (quantidade de processos)

# Criando o gráfico de bolhas
plt.figure(figsize=(10, 6))
plt.scatter(x, y, s=size, alpha=0.5, c='blue', edgecolors='w', linewidth=0.5)

# Títulos e rótulos
plt.title('Dispersão de Tempo de Recurso Interno por Quantidade de Processos', fontsize=14)
plt.xlabel('Tempo de Recurso Interno (dias)', fontsize=12)
plt.ylabel('Quantidade de Processos', fontsize=12)

# Exibir o gráfico
plt.show()

Tempo de tramitação dos processos decididos em 2025

In [0]:
%sql
SELECT
    classe,
    ROUND(AVG(tempo_tramitacao), 0)                           AS media_tempo_tramitacao_dias,
    percentile_approx(tempo_tramitacao, 0.5)                  AS mediana_tempo_tramitacao_dias,
    COUNT(DISTINCT processo)                                  AS qtde_processos
FROM gold.vw_distribuicoes2025_decisoes
WHERE tempo_tramitacao IS NOT NULL
  AND YEAR(data_baixa) = 2025
GROUP BY classe
ORDER BY qtde_processos DESC;

In [0]:

import matplotlib.pyplot as plt

# 1) Buscar dados (tempo_tramitacao) apenas das 5 classes mais frequentes, decididos em 2025
query = """
WITH top_classes AS (
    SELECT
        classe,
        COUNT(DISTINCT processo) AS qtde_processos
    FROM gold.vw_distribuicoes2025_decisoes
    WHERE tempo_tramitacao IS NOT NULL
      AND YEAR(data_baixa) = 2025
    GROUP BY classe
    ORDER BY qtde_processos DESC
    LIMIT 5
)
SELECT
    d.classe,
    d.tempo_tramitacao
FROM gold.vw_distribuicoes2025_decisoes d
JOIN top_classes t
  ON d.classe = t.classe
WHERE d.tempo_tramitacao IS NOT NULL
  AND YEAR(d.data_baixa) = 2025
"""

pdf = spark.sql(query).toPandas()

# 2) Ordenar as classes pelo volume (opcional, mas ajuda na leitura)
#    (se quiser, pode pegar a ordem do top_classes por outra query)
classes = sorted(pdf["classe"].unique())

# 3) Preparar os dados por classe
data_por_classe = [pdf.loc[pdf["classe"] == c, "tempo_tramitacao"].values for c in classes]

# 4) Plotar boxplot
plt.figure(figsize=(12, 7))
plt.boxplot(data_por_classe, labels=classes, showfliers=True)

plt.yscale("log")

plt.xlabel("Classe processual")
plt.ylabel("Tempo de tramitação (dias)")
plt.title("Boxplot do tempo de tramitação (processos baixados em 2025)\n5 classes com maior quantidade de processos")

plt.show()


Tempo de análise para o Relator proferir decisão final

In [0]:
%sql
WITH base AS (
    SELECT DISTINCT
        processo,
        classe,
        relator_atual_primeira AS relator,
        tempo_tramitacao,
        tipo_decisao_primeira 
    FROM gold.vw_distribuicoes2025_decisoes
    WHERE tempo_tramitacao IS NOT NULL
    AND tipo_decisao_primeira = "Decisão Final"
   
),
top_classes AS (
    SELECT
        classe
    FROM base
    GROUP BY classe
    ORDER BY COUNT(*) DESC
    LIMIT 5
)
SELECT
    b.relator,
    ROUND(AVG(b.tempo_tramitacao), 0) AS media_dias_ate_baixa,
    COUNT(*)                          AS qtde_processos
FROM base b
JOIN top_classes t
  ON b.classe = t.classe
GROUP BY b.relator
ORDER BY media_dias_ate_baixa, qtde_processos DESC;


#### Julgados favoráveis e desfavoráveis

In [0]:
%sql
CREATE OR REPLACE VIEW gold.vw_resultado_decisao_final_2025 AS
WITH finais AS (
    SELECT
        processo,
        data_decisao,
        relator_atual,
        orgao_julgador,
        assuntos_processo,
        andamento_decisao,
        ROW_NUMBER() OVER (
            PARTITION BY processo
            ORDER BY data_decisao DESC
        ) AS rn
    FROM silver.decisoes2025stf
    WHERE tipo_decisao = 'Decisão Final'
      AND data_decisao IS NOT NULL
),
final_unica AS (
    SELECT *
    FROM finais
    WHERE rn = 1
),
classificada AS (
    SELECT
        processo,
        data_decisao  AS data_decisao_final,
        relator_atual AS relator_decisao_final,
        orgao_julgador,
        assuntos_processo,
        andamento_decisao,

        CASE
            WHEN lower(coalesce(andamento_decisao,'')) LIKE '%segredo de justiça%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%sigilos%' THEN 'sigiloso'

            WHEN lower(coalesce(andamento_decisao,'')) LIKE '%homolog%' THEN 'homologacao'

            WHEN lower(coalesce(andamento_decisao,'')) LIKE '%devolv%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%retorno%' THEN 'devolucao'

            WHEN lower(coalesce(andamento_decisao,'')) LIKE '%conhecido%provido%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%provido em parte%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%parcialmente provido%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%provido%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%concedida%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%procedente%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%extinção da punibilidade%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%extincao da punibilidade%'
              OR lower(coalesce(andamento_decisao,'')) LIKE '%recebid%' THEN 'favoravel'

            ELSE 'desfavoravel'
        END AS resultado_final_classificado
    FROM final_unica
),
dist_unica AS (
    /* Garante 1 linha por processo para evitar duplicidade no join */
    SELECT
        processo,
        classe,
        orgao_origem,
        procedencia,
        ramo_direito
    FROM (
        SELECT
            processo,
            classe,
            orgao_origem,
            procedencia,
            ramo_direito,
            ROW_NUMBER() OVER (
                PARTITION BY processo
                ORDER BY data_autuacao DESC
            ) AS rn
        FROM silver.distribuicoes2025stf
    ) x
    WHERE rn = 1
)
SELECT
    c.*,
    d.classe,
    /* origem = tribunal/órgão de origem do processo */
    COALESCE(d.orgao_origem, d.procedencia) AS tribunal_origem,
    d.orgao_origem,
    d.procedencia,
    d.ramo_direito
FROM classificada c
LEFT JOIN dist_unica d
  ON c.processo = d.processo;



In [0]:
%sql
SELECT
    resultado_final_classificado,
    COUNT(DISTINCT processo) AS qtde_processos
FROM gold.vw_resultado_decisao_final_2025
GROUP BY resultado_final_classificado
ORDER BY qtde_processos DESC;


In [0]:
%sql
SELECT
    orgao_julgador,
    ambiente_julgamento,
    resultado_final_classificado,
    COUNT(DISTINCT processo) AS qtde
FROM gold.vw_resultado_decisao_final_2025
GROUP BY orgao_julgador, resultado_final_classificado, ambiente_julgamento
ORDER BY orgao_julgador, qtde DESC;



In [0]:
%sql
WITH base AS (
    SELECT
        procedencia,
        resultado_final_classificado,
        COUNT(DISTINCT processo) AS qtde_processos
    FROM gold.vw_resultado_decisao_final_2025
    GROUP BY procedencia, resultado_final_classificado
)
SELECT
    procedencia,
    resultado_final_classificado,
    qtde_processos,
    SUM(qtde_processos) OVER (PARTITION BY procedencia) AS total_tribunal,
    ROUND(
        100.0 * qtde_processos / SUM(qtde_processos) OVER (PARTITION BY procedencia),
        2
    ) AS percentual_no_tribunal
FROM base
ORDER BY qtde_processos DESC;





###Quantidade de processos distribuídos em 2025 que possuem ou não decisão

#### 6. Auto avaliação 